In [1]:
# !sudo apt install redis-server
# !pip3 install redis

In [2]:
from glob import glob
from collections import defaultdict
from tqdm import tqdm

files = sorted(glob('splitted/*.split'))
len(files)

110

In [3]:
warcs = defaultdict(set)

In [4]:
for f in tqdm(files[:1]):
    with open(f) as fopen:
        for line in fopen:
            d = line.rstrip()
            splitted = d.split()
            combined = f'{splitted[1]}-{splitted[3]}'
            warcs[splitted[0]].add(combined)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.40s/it]


In [5]:
for k in warcs:
    warcs[k] = list(warcs[k])

In [6]:
warcs_list = []
for k, v in warcs.items():
    warcs_list.append({'k': k, 'v': v})

In [7]:
warcs_list[0]

{'k': 'crawl-data/CC-MAIN-2018-05/segments/1516084890394.46/wet/CC-MAIN-20180121080507-20180121100507-00117.warc.wet.gz',
 'v': ['sha1:A7A24L3367EOT4WMTZ2BOYFGAPXIVDW3-764']}

In [59]:
import json

with open('warcs-eng_Latn-zsm_Latn.jsonl', 'w') as fopen:
    for i in range(len(warcs_list)):
        fopen.write(json.dumps(warcs_list[i]) + '\n')

In [24]:
import tempfile
import requests
import warc
import redis
from unidecode import unidecode
from tqdm import tqdm

def loop(files):
    files, _ = files
    db = redis.Redis(host='localhost', port=6379, db=0)
    for f in tqdm(files):
        k = f['k']
        v = f['v']
        
        if db.get(k):
            continue
            
        digest = {i.split('-')[0]: i.split('-')[1] for i in v}
        with tempfile.NamedTemporaryFile(suffix = '.warc.wet.gz') as tmpfile:
            url = f'http://data.commoncrawl.org/{k}'
            r = requests.get(url)
            tmpfile.write(r.content)
            f = warc.open(tmpfile.name)
            for record in f:
                if 'warc-block-digest' in record.header and record.header['warc-block-digest'] in digest:
                    r = record.payload.read()
                    sha = record.header['warc-block-digest']
                    line_num = int(digest[sha])
                    lines = unidecode(r.decode().split('\n')[line_num])
                    key = f'{k}-{sha}-{line_num}'
                    db.set(key, lines)
        
        db.set(k, 'true')
        
    db.close()

In [25]:
import mp

In [ ]:
mp.multiprocessing(warcs_list, loop, cores = 15, returned = False)

  0%|                                                                                          | 1/35533 [00:38<384:06:03, 38.92s/it]

In [73]:
loop((warcs_list[:2], 0))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:25<00:00, 12.93s/it]


In [11]:
db = redis.Redis(host='localhost', port=6379, db=0)

In [23]:
db.get('1')

In [21]:
keys = db.keys()

In [13]:
len(db.keys())

95

In [16]:
with open(files[0]) as fopen:
    data = fopen.read().split('\n')

In [19]:
splitted = data[3].split()
key = f'{splitted[0]}-{splitted[1]}-{splitted[3]}'

In [20]:
db.get(key)

b'a thousand disbelievers because the disbelievers are a people who do not have'

In [18]:
db.get(key)